In [1]:
import random
import cv2
import time
import argparse
import copy

import metaworld
import matplotlib.pyplot as plt
import numpy as np
import d3rlpy

from data_collection.random_sample import visualize, END_ITER_KEY

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
ml1 = metaworld.ML1("pick-place-v2")
env = ml1.train_classes["pick-place-v2"]()
env.set_task(ml1.train_tasks[0])

eval_env = ml1.train_classes["pick-place-v2"](render_mode="rgb_array")
eval_env.set_task(ml1.train_tasks[0])
eval_env.max_path_length = 500

In [4]:
td3 = d3rlpy.algos.TD3Config().create(device='cpu')
explorer = d3rlpy.algos.ConstantEpsilonGreedy(epsilon=0.3)
buffer = d3rlpy.dataset.create_fifo_replay_buffer(limit=50000, env=env)

2023-11-10 00:56:25 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('float64')], shape=[(4,)]) observation_signature=Signature(dtype=[dtype('float64')], shape=[(39,)]) reward_signature=Signature(dtype=[dtype('float32')], shape=[[1]])
2023-11-10 00:56:25 [info     ] Action-space has been automatically determined. action_space=<ActionSpace.CONTINUOUS: 1>
2023-11-10 00:56:25 [info     ] Action size has been automatically determined. action_size=4


In [5]:
td3.fit_online(
    env,
    buffer,
    explorer,
    eval_env=eval_env,
    n_steps=5000,
    n_steps_per_epoch=1000,
)

2023-11-10 00:56:25 [info     ] Directory is created at d3rlpy_logs/TD3_online_20231110005625
2023-11-10 00:56:25 [debug    ] Building model...
2023-11-10 00:56:25 [debug    ] Model has been built.
2023-11-10 00:56:25 [info     ] Parameters                     params={'observation_shape': [39], 'action_size': 4, 'config': {'type': 'td3', 'params': {'batch_size': 256, 'gamma': 0.99, 'observation_scaler': {'type': 'none', 'params': {}}, 'action_scaler': {'type': 'none', 'params': {}}, 'reward_scaler': {'type': 'none', 'params': {}}, 'actor_learning_rate': 0.0003, 'critic_learning_rate': 0.0003, 'actor_optim_factory': {'type': 'adam', 'params': {'betas': [0.9, 0.999], 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}}, 'critic_optim_factory': {'type': 'adam', 'params': {'betas': [0.9, 0.999], 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}}, 'actor_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'critic_encoder_fa

  0%|          | 0/5000 [00:00<?, ?it/s]

2023-11-10 00:56:36 [info     ] Model parameters are saved to d3rlpy_logs/TD3_online_20231110005625/model_1000.d3
2023-11-10 00:56:36 [info     ] TD3_online_20231110005625: epoch=1 step=1000 epoch=1 metrics={'time_inference': 0.0002105123996734619, 'time_environment_step': 0.0011194000244140626, 'time_step': 0.006263592720031738, 'time_sample_batch': 0.0015679779155418016, 'time_algorithm_update': 0.005029734820729149, 'critic_loss': 6.33268181302383e-05, 'actor_loss': 0.05676862233949284, 'rollout_return': 2.3605031335149107, 'evaluation': 3.255986601212231} step=1000
2023-11-10 00:56:49 [info     ] Model parameters are saved to d3rlpy_logs/TD3_online_20231110005625/model_2000.d3
2023-11-10 00:56:49 [info     ] TD3_online_20231110005625: epoch=2 step=2000 epoch=2 metrics={'time_inference': 0.00021330785751342773, 'time_environment_step': 0.0011270804405212403, 'time_sample_batch': 0.0015907621383666993, 'time_algorithm_update': 0.004674025535583496, 'critic_loss': 1.0825207514471913e-

In [6]:
truncate = False
terminal = False
obs, _ = eval_env.reset()
visualize(eval_env.render()[:,:,::-1])
while not truncate and not terminal:
    keystroke = cv2.waitKey(0)
    if keystroke == ord(END_ITER_KEY):
        break
    elif keystroke == ord(" "):
        action = td3.predict(obs[None, :])[0]
        obs, reward, terminal, truncate, info = eval_env.step(action)
    visualize(eval_env.render()[:,:,::-1])
    if info['success']:
        print('the task is successful!')
        break
cv2.destroyAllWindows()

In [8]:
eval_env.render()

array([[[127, 126, 122],
        [127, 126, 122],
        [127, 126, 122],
        ...,
        [127, 126, 122],
        [127, 126, 122],
        [127, 126, 122]],

       [[127, 126, 122],
        [127, 126, 122],
        [127, 126, 122],
        ...,
        [127, 126, 122],
        [127, 126, 122],
        [127, 126, 122]],

       [[127, 126, 122],
        [127, 126, 122],
        [127, 126, 122],
        ...,
        [127, 126, 122],
        [127, 126, 122],
        [127, 126, 122]],

       ...,

       [[219, 221, 221],
        [220, 221, 221],
        [221, 221, 221],
        ...,
        [215, 216, 216],
        [216, 216, 216],
        [217, 218, 218]],

       [[222, 222, 222],
        [223, 223, 223],
        [224, 225, 224],
        ...,
        [212, 214, 214],
        [214, 215, 215],
        [216, 216, 216]],

       [[224, 224, 224],
        [224, 224, 224],
        [224, 224, 224],
        ...,
        [214, 215, 215],
        [216, 217, 216],
        [217, 218, 218]]